In [1]:
import json

In [8]:
with open("cabral_full_catalog.html_scrape.json", "r") as f:
    col = json.load(f)

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import time
import json
from urllib.parse import urljoin
from requests.exceptions import RequestException

BASE_URL = "https://cabraloutdoors.com"

#––– Helpers –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

def get_soup(url):
    """Fetch a URL and return BeautifulSoup object, with error handling."""
    try:
        resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, timeout=10)
        resp.raise_for_status()
        return BeautifulSoup(resp.text, "html.parser")
    except RequestException as e:
        print(f"[ERROR] Failed to fetch URL: {url}\n{e}")
        return None


c:\Users\kggau\anaconda3\envs\streamlitnew\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.4.0) or chardet (5.1.0)/charset_normalizer (2.1.1) doesn't match a supported version!
  warnings.warn(


In [3]:
col_url = f"https://cabraloutdoors.com/collections/frogs/products/z-man-hard-leg-frogz-4inch-3-pack-10x-tough-super-amphibian"
soup = get_soup(col_url)

In [ ]:
soup.find("div", class_="jdgm-prev-badge")

<div class="jdgm-prev-badge" data-average-rating="4.79" data-number-of-questions="13" data-number-of-reviews="81" style="display:none"> <span aria-label="4.79 stars" class="jdgm-prev-badge__stars" data-score="4.79" role="button" tabindex="0"> <span class="jdgm-star jdgm--on"></span><span class="jdgm-star jdgm--on"></span><span class="jdgm-star jdgm--on"></span><span class="jdgm-star jdgm--on"></span><span class="jdgm-star jdgm--on"></span> </span> <span class="jdgm-prev-badge__text"> 81 reviews </span> </div>

In [6]:
soup.find("div", class_="jdgm-prev-badge").get("data-average-rating")

'4.79'

In [7]:
soup.find("div", class_="jdgm-prev-badge").get("data-number-of-reviews")

'81'

In [1]:
import json
import time
import requests
from bs4 import BeautifulSoup
from requests.exceptions import RequestException

# Constants
INPUT_JSON = "cabral_full_catalog.html_scrape.json"
OUTPUT_JSON = "cabral_full_catalog_with_ratings11.json"
BASE_HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept-Language": "en-US,en;q=0.9"
}

c:\Users\kggau\anaconda3\envs\streamlitnew\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.4.0) or chardet (5.1.0)/charset_normalizer (2.1.1) doesn't match a supported version!
  warnings.warn(


In [2]:
def get_soup(url):
    """Fetch a URL and return a BeautifulSoup object, or None on failure."""
    try:
        resp = requests.get(url, headers=BASE_HEADERS, timeout=10)
        resp.raise_for_status()
        return BeautifulSoup(resp.text, "html.parser")
    except RequestException as e:
        print(f"[ERROR] Could not fetch {url}: {e}")
        return None

def scrape_ratings_summary(soup):
    """
    From a BeautifulSoup-parsed product page, extract:
      - average_rating
      - count_reviews
    Returns a dict with those two keys (values may be None).
    """
    badge = soup.find("div", class_="jdgm-prev-badge")
    if not badge:
        return {"average_rating": None, "count_reviews": None}

    return {
        "average_rating": badge.get("data-average-rating"),
        "count_reviews": badge.get("data-number-of-reviews")
    }